In [1]:
from datasets import load_dataset, Dataset
import pandas as pd


full_dataset_train = load_dataset("PKU-Alignment/PKU-SafeRLHF-30k", split="train")

full_dataset_test = load_dataset("PKU-Alignment/PKU-SafeRLHF-30k", split="test")

In [2]:
def get_harmhelp_df(dataset :  Dataset):
    df = dataset.to_pandas()
    rows = []
    for idx, row in df.iterrows():
        if row["is_response_0_safe"] ==  False and row["is_response_1_safe"] == False:
            if row["better_response_id"] == 0:
                rows.append({
                    "prompt" : row["prompt"],
                    "chosen": row["response_0"],
                    "rejected" : row["response_1"],
                    "flag" : "unsafe-unsafe"
                })
            
            else:
                rows.append({
                    "prompt" : row["prompt"],
                    "chosen": row["response_1"],
                    "rejected" : row["response_0"],
                    "flag" : "unsafe-unsafe"
                })
    harmful_df = pd.DataFrame(rows)
    rows = []
    for idx, row in df.iterrows():
        if row["is_response_0_safe"] ==  True and row["is_response_1_safe"] ==  False and  row["better_response_id"] == 1:
                rows.append({
                    "prompt" : row["prompt"],
                    "chosen": row["response_0"],
                    "rejected" : row["response_1"],
                    "flag" : "safe-unsafe"
                })
        if row["is_response_0_safe"] ==  False and row["is_response_1_safe"] ==  True and  row["better_response_id"] == 0:
            rows.append({
                    "prompt" : row["prompt"],
                    "chosen": row["response_1"],
                    "rejected" : row["response_0"],
                    "flag" : "safe-unsafe"
                })

    helpful_df  = pd.DataFrame(rows)
    print(len(harmful_df) , len(helpful_df))
    return pd.concat([harmful_df, helpful_df], axis=0)
    
    
    
    

In [3]:
train_df = get_harmhelp_df(full_dataset_train)
test_df= get_harmhelp_df(full_dataset_test)

12934 3117
1368 355


In [4]:
train_df.to_csv("reward_train_data-new.csv", index=False)
test_df.to_csv("reward_test_data-new.csv", index= False)